<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Time series exercises

--- 

<h1>Lab Guide<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Load-the-CRSP-daily-returns.-Fit-an-ARMA-model-to-the-GE-column-and-tune-the-parameters." data-toc-modified-id="1.-Load-the-CRSP-daily-returns.-Fit-an-ARMA-model-to-the-GE-column-and-tune-the-parameters.-1">1. Load the CRSP daily returns. Fit an ARMA model to the GE column and tune the parameters.</a></span><ul class="toc-item"><li><span><a href="#Bonus:-Fit-also-the-other-columns." data-toc-modified-id="Bonus:-Fit-also-the-other-columns.-1.1">Bonus: Fit also the other columns.</a></span></li><li><span><a href="#Obtain-a-date-index-by-combining-the-year,-month-and-day-data." data-toc-modified-id="Obtain-a-date-index-by-combining-the-year,-month-and-day-data.-1.2">Obtain a date index by combining the year, month and day data.</a></span></li><li><span><a href="#Plot-the-timeseries-and-the-(partial)-autocorrelations" data-toc-modified-id="Plot-the-timeseries-and-the-(partial)-autocorrelations-1.3">Plot the timeseries and the (partial) autocorrelations</a></span></li><li><span><a href="#Fit-an-ARIMA-model" data-toc-modified-id="Fit-an-ARIMA-model-1.4">Fit an ARIMA model</a></span></li><li><span><a href="#Do-the-same-with-the-CRSP-data" data-toc-modified-id="Do-the-same-with-the-CRSP-data-1.5">Do the same with the CRSP data</a></span></li></ul></li><li><span><a href="#Load-the-Mishkin-dataset." data-toc-modified-id="Load-the-Mishkin-dataset.-2">Load the Mishkin dataset.</a></span><ul class="toc-item"><li><span><a href="#Bonus:-Use-also-the-columns-on-one-month-inflation-rate-(pai1)-and-consumer-price-index-for-urban-consumers-(cpi)." data-toc-modified-id="Bonus:-Use-also-the-columns-on-one-month-inflation-rate-(pai1)-and-consumer-price-index-for-urban-consumers-(cpi).-2.1">Bonus: Use also the columns on one-month inflation rate (pai1) and consumer price index for urban consumers (cpi).</a></span></li><li><span><a href="#Plot-the-time-series-for-the-tb1-data-before-and-after-taking-the-logarithm,-and-plot-it-together-with-the-one-or-two-times-differenced-timeseries" data-toc-modified-id="Plot-the-time-series-for-the-tb1-data-before-and-after-taking-the-logarithm,-and-plot-it-together-with-the-one-or-two-times-differenced-timeseries-2.2">Plot the time series for the tb1 data before and after taking the logarithm, and plot it together with the one or two times differenced timeseries</a></span></li><li><span><a href="#Plot-the-(partial)-autocorrelations-before-and-after-differencing-and-compare" data-toc-modified-id="Plot-the-(partial)-autocorrelations-before-and-after-differencing-and-compare-2.3">Plot the (partial) autocorrelations before and after differencing and compare</a></span></li><li><span><a href="#Select-the-best-model-order-according-to-AIC-and-cross-validation-and-compare" data-toc-modified-id="Select-the-best-model-order-according-to-AIC-and-cross-validation-and-compare-2.4">Select the best model order according to AIC and cross validation and compare</a></span></li><li><span><a href="#With-external-predictors" data-toc-modified-id="With-external-predictors-2.5">With external predictors</a></span></li></ul></li></ul></div>

In [1]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
sns.set(font_scale=1.5)
%matplotlib inline

sns.set(font_scale=1.5)
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### 1. Load the CRSP daily returns. Fit an ARMA model to the GE column and tune the parameters.

#### Bonus: Fit also the other columns.

#### Obtain a date index by combining the year, month and day data.

In [2]:
data = sm.datasets.get_rdataset('CRSPday', package='Ecdat')

/Users/crahmede/anaconda3/lib/python3.7/site-packages/statsmodels/datasets/utils.py:192: FutureWarning: `item` has been deprecated and will be removed in a future version
  return dataset_meta["Title"].item()


In [3]:
print(data.__doc__)

+---------+-----------------+
| CRSPday | R Documentation |
+---------+-----------------+

Daily Returns from the CRSP Database
------------------------------------

Description
~~~~~~~~~~~

daily observations from 1969-1-03 to 1998-12-31

*number of observations* : 2528

*observation* : production units

*country* : United States

Usage
~~~~~

::

   data(CRSPday)

Format
~~~~~~

A dataframe containing :

year
   the year

month
   the month

day
   the day

ge
   the return for General Electric, Permno 12060

ibm
   the return for IBM, Permno 12490

mobil
   the return for Mobil Corporation, Permno 15966

crsp
   the return for the CRSP value-weighted index, including dividends

Source
~~~~~~

Center for Research in Security Prices, Graduate School of Business,
University of Chicago, 725 South Wells - Suite 800, Chicago, Illinois
60607, http://www.crsp.com.

References
~~~~~~~~~~

Davidson, R. and James G. MacKinnon (2004) *Econometric Theory and
Methods*, New York, Oxford University

In [4]:
df = data.data
df.head()

,year,month,day,ge,ibm,mobil,crsp
0,1989,1,3,-0.016760,0.000000,-0.002747,-0.007619
1,1989,1,4,0.017045,0.005128,0.005510,0.013016
2,1989,1,5,-0.002793,-0.002041,0.005479,0.002815
3,1989,1,6,0.000000,-0.006135,0.002725,0.003064
4,1989,1,9,0.000000,0.004115,0.005435,0.001633


#### Plot the timeseries and the (partial) autocorrelations

Compare before and after differencing. Do you think that your model will require differencing?

#### Fit an ARIMA model

1. Use automatic selection based on AIC to select the best model order
2. Use cross validation to select the best model order and compare

#### Do the same with the CRSP data

### Load the Mishkin dataset. 

- Consider the time series of the logarithm of the tb1 (monthly one-month T-bill rates) values. 
- Judge from autocorrelation plots how much differencing is required to obtain a stationary time series. 
- Fit an ARMA model to the resulting time series. 

#### Bonus: Use also the columns on one-month inflation rate (pai1) and consumer price index for urban consumers (cpi).

In [17]:
data_1 = sm.datasets.get_rdataset('Mishkin', package='Ecdat')

/Users/crahmede/anaconda3/lib/python3.7/site-packages/statsmodels/datasets/utils.py:192: FutureWarning: `item` has been deprecated and will be removed in a future version
  return dataset_meta["Title"].item()


In [18]:
print(data_1.__doc__)

+---------+-----------------+
| Mishkin | R Documentation |
+---------+-----------------+

Inflation and Interest Rates
----------------------------

Description
~~~~~~~~~~~

monthly observations from 1950-2 to 1990-12

*number of observations* : 491

*observation* : country

*country* : United States

Usage
~~~~~

::

   data(Mishkin)

Format
~~~~~~

A time serie containing :

pai1
   one-month inflation rate (in percent, annual rate)

pai3
   three-month inflation rate (in percent, annual rate)

tb1
   one-month T-bill rate (in percent, annual rate)

tb3
   three-month T-bill rate (in percent, annual rate)

cpi
   CPI for urban consumers, all items (the 1982-1984 average is set to
   100)

Source
~~~~~~

Mishkin, F. (1992) “Is the Fisher effect for real ?”, *Journal of
Monetary Economics*, **30**, 195-215.

References
~~~~~~~~~~

Hayashi, F. (2000) *Econometrics*, Princeton University Press,
http://fhayashi.fc2web.com/hayashi_econometrics.htm, chapter 2, 176-184.

See Also
~~~~~~~~



In [19]:
dates = pd.period_range(start='1950-02-01', end='1990-12-01', freq='M')

In [20]:
df_1 = data_1.data
df_1.index = dates
df_1['tb1log'] = np.log(df_1.tb1)
df_1.head()

,pai1,pai3,tb1,tb3,cpi,tb1log
1950-02,-3.552289,1.129370,1.100854,1.129406,23.5,0.096086
1950-03,5.247540,4.001566,1.125513,1.137254,23.6,0.118239
1950-04,1.692860,4.492160,1.115715,1.142319,23.6,0.109495
1950-05,5.064298,7.817513,1.146380,1.177902,23.7,0.136609
1950-06,6.719322,9.433580,1.158520,1.167777,23.8,0.147143


#### Plot the time series for the tb1 data before and after taking the logarithm, and plot it together with the one or two times differenced timeseries

#### Plot the (partial) autocorrelations before and after differencing and compare

#### Select the best model order according to AIC and cross validation and compare